# Creating, Sending and Receiving Packets

In [ ]:
from scapy.all import *

In [ ]:
conf.verb=1
conf.color_theme = RastaTheme()

## Stacking Layers

- the `/` operator is used as a composition operator between two layers
- the lower layer can have one or more of its defaults fields overloaded according to the upper layer
    - you still can give the value you want
    - a string can be used as a raw layer
    
<img src="http://scapy.readthedocs.io/en/latest/_images/fieldsmanagement.png" />

In [4]:
IP().show()

###[ IP ]###
  version   = 4
  ihl       = None
  tos       = 0x0
  len       = None
  id        = 1
  flags     = 
  frag      = 0
  ttl       = 64
  proto     = hopopt
  chksum    = None
  src       = 127.0.0.1
  dst       = 127.0.0.1
  \options   \



In [6]:
TCP()/IP()/TCP()

<TCP  |<IP  frag=0 proto=tcp |<TCP  |>>>

In [7]:
Ether()/IP()/TCP()

<Ether  type=IPv4 |<IP  frag=0 proto=tcp |<TCP  |>>>

In [8]:
Ether()/IP()/TCP()/"GET / HTTP/1.0\r\n\r\n"

<Ether  type=IPv4 |<IP  frag=0 proto=tcp |<TCP  |<Raw  load=b'GET / HTTP/1.0\r\n\r\n' |>>>>

In [9]:
Ether()/IP()/IP()/UDP()

<Ether  type=IPv4 |<IP  frag=0 proto=ipencap |<IP  frag=0 proto=udp |<UDP  |>>>>

In [12]:
IP(proto=55)/TCP(dport=80)

<IP  frag=0 proto=55 |<TCP  dport=http |>>

## Build ARP packets
- build ARP packet which has two layers - Ether and ARP

In [13]:
ls(Ether)

dst        : DestMACField                        = ('None')
src        : SourceMACField                      = ('None')
type       : XShortEnumField                     = ('36864')


In [14]:
ls(ARP)

hwtype     : XShortEnumField                     = ('1')
ptype      : XShortEnumField                     = ('2048')
hwlen      : FieldLenField                       = ('None')
plen       : FieldLenField                       = ('None')
op         : ShortEnumField                      = ('1')
hwsrc      : MultipleTypeField (SourceMACField, StrFixedLenField) = ('None')
psrc       : MultipleTypeField (SourceIPField, SourceIP6Field, StrFixedLenField) = ('None')
hwdst      : MultipleTypeField (MACField, StrFixedLenField) = ('None')
pdst       : MultipleTypeField (IPField, IP6Field, StrFixedLenField) = ('None')


In [15]:
arpPkt = Ether()/ARP()

In [17]:
arpPkt.show()

###[ Ethernet ]###
  dst       = None
  src       = 00:0c:29:de:4e:98
  type      = ARP
###[ ARP ]###
     hwtype    = Ethernet (10Mb)
     ptype     = IPv4
     hwlen     = None
     plen      = None
     op        = who-has
     hwsrc     = 00:0c:29:de:4e:98
     psrc      = 192.168.35.128
     hwdst     = 00:00:00:00:00:00
     pdst      = 0.0.0.0



In [18]:
# Change Ethernet protocol's destination field to broadcase MAC address
arpPkt['Ethernet'].dst= 'FF:FF:FF:FF:FF:FF'

In [20]:
arpPkt.show()

###[ Ethernet ]###
  dst       = FF:FF:FF:FF:FF:FF
  src       = 00:0c:29:de:4e:98
  type      = ARP
###[ ARP ]###
     hwtype    = Ethernet (10Mb)
     ptype     = IPv4
     hwlen     = None
     plen      = None
     op        = who-has
     hwsrc     = 00:0c:29:de:4e:98
     psrc      = 192.168.35.128
     hwdst     = 00:00:00:00:00:00
     pdst      = 0.0.0.0



In [21]:
# change ARP protocol's hwdst to broadcast MAC address
#arpPkt['Ethernet']['ARP'].hwdst = 'FF:FF:FF:FF:FF:FF'
arpPkt['ARP'].hwdst = 'FF:FF:FF:FF:FF:FF'

In [23]:
! arp -n

Address                  HWtype  HWaddress           Flags Mask            Iface
192.168.35.254           ether   00:50:56:f0:96:f0   C                     eth0
192.168.35.2             ether   00:50:56:ff:0f:b0   C                     eth0


In [22]:
arpPkt.show()

###[ Ethernet ]###
  dst       = FF:FF:FF:FF:FF:FF
  src       = 00:0c:29:de:4e:98
  type      = ARP
###[ ARP ]###
     hwtype    = Ethernet (10Mb)
     ptype     = IPv4
     hwlen     = None
     plen      = None
     op        = who-has
     hwsrc     = 00:0c:29:de:4e:98
     psrc      = 192.168.35.128
     hwdst     = FF:FF:FF:FF:FF:FF
     pdst      = 0.0.0.0



In [24]:
# change dest ip - use Metaploitable IP
arpPkt['ARP'].pdst = '192.168.35.130'

In [25]:
arpPkt.show()

###[ Ethernet ]###
  dst       = FF:FF:FF:FF:FF:FF
  src       = 00:0c:29:de:4e:98
  type      = ARP
###[ ARP ]###
     hwtype    = Ethernet (10Mb)
     ptype     = IPv4
     hwlen     = None
     plen      = None
     op        = who-has
     hwsrc     = 00:0c:29:de:4e:98
     psrc      = 192.168.35.128
     hwdst     = FF:FF:FF:FF:FF:FF
     pdst      = 192.168.35.130



## Capture packet using wireshark

- ARP packet is ready to be sent
- Makesure Ethernet source is selected
- Open Wireshark and start capturing traffic
- use arp filter to display only ARP packets

# Send packets - one way
- sendp() - works at layer two; can send ARP, Ethernet, etc.
- send() - send packets at layer 3 (IP, TCP, HTTP, etc.); will handle routing and layer 2
    - if return_packets=True is passed as parameter, these functions will also return received packes

In [26]:
help(send)

Help on function send in module scapy.sendrecv:

send(x, **kargs)
    Send packets at layer 3

    This determines the interface (or L2 source to use) based on the routing
    table: conf.route / conf.route6

    :param x: the packets
    :param inter: time (in s) between two packets (default 0)
    :param loop: send packet indefinitely (default 0)
    :param count: number of packets to send (default None=1)
    :param verbose: verbose mode (default None=conf.verb)
    :param realtime: check that a packet was sent before sending the next one
    :param return_packets: return the sent packets
    :param socket: the socket to use (default is conf.L3socket(kargs))
    :param monitor: (not on linux) send in monitor mode
    :returns: None



In [27]:
help(sendp)

Help on function sendp in module scapy.sendrecv:

sendp(x, iface=None, iface_hint=None, socket=None, **kargs)
    Send packets at layer 2

    :param x: the packets
    :param inter: time (in s) between two packets (default 0)
    :param loop: send packet indefinitely (default 0)
    :param count: number of packets to send (default None=1)
    :param verbose: verbose mode (default None=conf.verb)
    :param realtime: check that a packet was sent before sending the next one
    :param return_packets: return the sent packets
    :param socket: the socket to use (default is conf.L3socket(kargs))
    :param iface: the interface to send the packets on
    :param monitor: (not on linux) send in monitor mode
    :returns: None



In [30]:
! arp -n

Address                  HWtype  HWaddress           Flags Mask            Iface
192.168.35.254           ether   00:50:56:f0:96:f0   C                     eth0
192.168.35.2             ether   00:50:56:ff:0f:b0   C                     eth0


In [28]:
# send the arpPkt - use sendp() because ARP is layer 2 packet
sendp(arpPkt)


Sent 1 packets.
.

In [31]:
response = srp1(arpPkt)

Begin emission

Finished sending 1 packets

Received 1 packets, got 1 answers, remaining 0 packets


In [32]:
response.show()

###[ Ethernet ]###
  dst       = 00:0c:29:de:4e:98
  src       = 00:0c:29:0b:ca:da
  type      = ARP
###[ ARP ]###
     hwtype    = Ethernet (10Mb)
     ptype     = IPv4
     hwlen     = 6
     plen      = 4
     op        = is-at
     hwsrc     = 00:0c:29:0b:ca:da
     psrc      = 192.168.35.130
     hwdst     = 00:0c:29:de:4e:98
     pdst      = 192.168.35.128
###[ Padding ]###
        load      = b'\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00'



In [33]:
# doesn't show IP just with ARP
! arp -n

Address                  HWtype  HWaddress           Flags Mask            Iface
192.168.35.254           ether   00:50:56:f0:96:f0   C                     eth0
192.168.35.2             ether   00:50:56:ff:0f:b0   C                     eth0


In [34]:
# one liner
sendp(Ether(dst="ff:ff:ff:ff:ff:ff",src="00:11:22:aa:bb:cc")/ARP(hwsrc="00:11:22:aa:bb:cc",pdst="192.168.231.2"))

.
Sent 1 packets.


## Sending HTTP request

- run a simple Flask Hello World Web app on another VM
- see `demos/FlaskWebApp` folder
- follow the instructions in README

In [49]:
# Metaploitable runs http server on port 80
# Most real-world websites use 443

httpPkt = IP(dst="192.168.35.130")/TCP(dport=5000)/"GET /index.html HTTP/1.0\n\n"

In [50]:
bytes(httpPkt)

b'E\x00\x00B\x00\x01\x00\x00@\x06\xb2b\xc0\xa8#\x80\xc0\xa8#\x82\x00\x14\x13\x88\x00\x00\x00\x00\x00\x00\x00\x00P\x02 \x00\x02(\x00\x00GET /index.html HTTP/1.0\n\n'

In [51]:
hexdump(httpPkt)

0000  45 00 00 42 00 01 00 00 40 06 B2 62 C0 A8 23 80  E..B....@..b..#.
0010  C0 A8 23 82 00 14 13 88 00 00 00 00 00 00 00 00  ..#.............
0020  50 02 20 00 02 28 00 00 47 45 54 20 2F 69 6E 64  P. ..(..GET /ind
0030  65 78 2E 68 74 6D 6C 20 48 54 54 50 2F 31 2E 30  ex.html HTTP/1.0
0040  0A 0A                                            ..


In [52]:
httpPkt.show()

###[ IP ]###
  version   = 4
  ihl       = None
  tos       = 0x0
  len       = None
  id        = 1
  flags     = 
  frag      = 0
  ttl       = 64
  proto     = tcp
  chksum    = None
  src       = 192.168.35.128
  dst       = 192.168.35.130
  \options   \
###[ TCP ]###
     sport     = ftp_data
     dport     = 5000
     seq       = 0
     ack       = 0
     dataofs   = None
     reserved  = 0
     flags     = S
     window    = 8192
     chksum    = None
     urgptr    = 0
     options   = []
###[ Raw ]###
        load      = b'GET /index.html HTTP/1.0\n\n'



## Send and Receive Packets
- Send and receive functions are the heart of Scapy
- sr() - send L3 packets and receive answers; returns answered and unanswered packets
- sr1() - returns only one packet that answered the sent packet
- sr() and sr1() functions can only send layer 3 L3 packets (IP, HTTP, etc.)
- srp() - send and receive L2 packets (ARP, Ehternet, 802.3, etc)
- srp1() - send and receive one layer 2 packet

In [53]:
response = sr(httpPkt)

Begin emission

Finished sending 1 packets

Received 2 packets, got 1 answers, remaining 0 packets


In [55]:
# get the answer tuple
ans = response[0]

In [45]:
help(ans)

Help on SndRcvList in module scapy.plist object:

class SndRcvList(_PacketList, scapy.base_classes.BasePacketList, scapy.base_classes._CanvasDumpExtended)
 |  SndRcvList(res=None, name='Results', stats=None)
 |
 |  Method resolution order:
 |      SndRcvList
 |      _PacketList
 |      scapy.base_classes.BasePacketList
 |      scapy.base_classes.Gen
 |      typing.Generic
 |      scapy.base_classes._CanvasDumpExtended
 |      builtins.object
 |
 |  Methods defined here:
 |
 |  __init__(self, res=None, name='Results', stats=None)
 |      create a packet list from a list of packets
 |      res: the list of packets
 |      stats: a list of classes that will appear in the stats (defaults to [TCP,UDP,ICMP])
 |
 |  ----------------------------------------------------------------------
 |  Data descriptors defined here:
 |
 |  __dict__
 |      dictionary for instance variables
 |
 |  __weakref__
 |      list of weak references to the object
 |
 |  ---------------------------------------------

In [56]:
ans.show()

0000 IP / TCP 192.168.35.128:ftp_data > 192.168.35.130:5000 S / Raw ==> IP / TCP 192.168.35.130:5000 > 192.168.35.128:ftp_data SA / Padding


In [57]:
print(ans[0])

QueryAnswer(query=<IP  frag=0 proto=tcp dst=192.168.35.130 |<TCP  dport=5000 |<Raw  load=b'GET /index.html HTTP/1.0\n\n' |>>>, answer=<IP  version=4 ihl=5 tos=0x0 len=44 id=0 flags=DF frag=0 ttl=64 proto=tcp chksum=0x7279 src=192.168.35.130 dst=192.168.35.128 |<TCP  sport=5000 dport=ftp_data seq=2248820217 ack=1 dataofs=6 reserved=0 flags=SA window=64240 chksum=0xf531 urgptr=0 options=[('MSS', 1460)] |<Padding  load=b'\x00\x00' |>>>)


In [58]:
ans[0]

QueryAnswer(query=<IP  frag=0 proto=tcp dst=192.168.35.130 |<TCP  dport=5000 |<Raw  load=b'GET /index.html HTTP/1.0\n\n' |>>>, answer=<IP  version=4 ihl=5 tos=0x0 len=44 id=0 flags=DF frag=0 ttl=64 proto=tcp chksum=0x7279 src=192.168.35.130 dst=192.168.35.128 |<TCP  sport=5000 dport=ftp_data seq=2248820217 ack=1 dataofs=6 reserved=0 flags=SA window=64240 chksum=0xf531 urgptr=0 options=[('MSS', 1460)] |<Padding  load=b'\x00\x00' |>>>)

## TCP/IP full three-way handshake

### send SYN packet

In [67]:
# Create syn packet
host = '192.168.35.130'
port = 5000
syn = IP(dst=f'{host}')/TCP(dport=port, flags='S')

In [68]:
syn.show()

###[ IP ]###
  version   = 4
  ihl       = None
  tos       = 0x0
  len       = None
  id        = 1
  flags     = 
  frag      = 0
  ttl       = 64
  proto     = tcp
  chksum    = None
  src       = 192.168.35.128
  dst       = 192.168.35.130
  \options   \
###[ TCP ]###
     sport     = ftp_data
     dport     = 5000
     seq       = 0
     ack       = 0
     dataofs   = None
     reserved  = 0
     flags     = S
     window    = 8192
     chksum    = None
     urgptr    = 0
     options   = []



In [69]:
# Receive SYN-ACK packet
syn_ack = sr1(syn)

Begin emission

Finished sending 1 packets

Received 1 packets, got 1 answers, remaining 0 packets


In [70]:
syn_ack.show()

###[ IP ]###
  version   = 4
  ihl       = 5
  tos       = 0x0
  len       = 44
  id        = 0
  flags     = DF
  frag      = 0
  ttl       = 64
  proto     = tcp
  chksum    = 0x7279
  src       = 192.168.35.130
  dst       = 192.168.35.128
  \options   \
###[ TCP ]###
     sport     = 5000
     dport     = ftp_data
     seq       = 1426801162
     ack       = 1
     dataofs   = 6
     reserved  = 0
     flags     = SA
     window    = 64240
     chksum    = 0x2a20
     urgptr    = 0
     options   = [('MSS', 1460)]
###[ Padding ]###
        load      = b'\x00\x00'



In [ ]:
# Send Get request; dosn't work but you get the gist of what should be happening
getStr = f'GET / HTTP1.1\r\nHost:{host}\r\n\r\n'
request = IP(dst=f'{host}') / TCP(dport=port, sport=syn_ack[TCP].dport, 
                                        seq=syn_ack[TCP].ack, ack=syn_ack[TCP].seq+1, 
                                        flags='A') / getStr

In [72]:
request.show()

###[ IP ]###
  version   = 4
  ihl       = None
  tos       = 0x0
  len       = None
  id        = 1
  flags     = 
  frag      = 0
  ttl       = 64
  proto     = tcp
  chksum    = None
  src       = 192.168.35.128
  dst       = 192.168.35.130
  \options   \
###[ TCP ]###
     sport     = ftp_data
     dport     = 5000
     seq       = 1
     ack       = 1426801163
     dataofs   = None
     reserved  = 0
     flags     = A
     window    = 8192
     chksum    = None
     urgptr    = 0
     options   = []
###[ Raw ]###
        load      = b'GET / HTTP1.1\r\nHost:192.168.35.130\r\n\r\n'



In [73]:
# send and get response
reply = sr1(request)

Begin emission

Finished sending 1 packets

Received 2 packets, got 1 answers, remaining 0 packets


In [ ]:
# Server sends RES reset packet instead of HTTP data with ACK flag
reply.show()

###[ IP ]###
  version   = 4
  ihl       = 5
  tos       = 0x0
  len       = 40
  id        = 0
  flags     = DF
  frag      = 0
  ttl       = 64
  proto     = tcp
  chksum    = 0x727d
  src       = 192.168.35.130
  dst       = 192.168.35.128
  \options   \
###[ TCP ]###
     sport     = 5000
     dport     = ftp_data
     seq       = 1426801163
     ack       = 0
     dataofs   = 5
     reserved  = 0
     flags     = R
     window    = 0
     chksum    = 0x3cdb
     urgptr    = 0
     options   = []
###[ Padding ]###
        load      = b'\x00\x00\x00\x00\x00\x00'



In [75]:
reply.summary()

'IP / TCP 192.168.35.130:5000 > 192.168.35.128:ftp_data R / Padding'

In [ ]:
tcpip = IP(dst="example.com") / TCP(dport=80) # tcp packet needs dest port
reply = sr(tcpip)
reply

In [ ]:
ans, unans = reply 

In [ ]:
ans.summary()

In [ ]:
unans

## TCP Syn Flooding
-  Not quite working with Scapy but should world with netwox
- form of DoS attack where attacker doesn't complete 3-way handshake by not sending ACK packet after receiving SYN-ACK from server
- attacker typically floods the victim's half-opened connections
- when queue is full, victim cannot take any more new connection
- See Taks 1 here: http://www.cis.syr.edu/~wedu/seed/Labs_16.04/Networking/TCP_Attacks/TCP_Attacks.pdf
- queue size can be viwed by using the following command

In [ ]:
# on a server VM, run the following command
# Run SEEDLab VM which has Apache server running
! sysctl -q net.ipv4.tcp_max_syn_backlog

In [ ]:
# set max backlog/queue size to 1
! sudo sysctl -w net.ipv4.tcp_max_syn_backlog=2

In [ ]:
# Turn of SYN cookie countermeasure
! sudo sysctl -a | grep cookie #(Display the SYN cookie flag)

In [ ]:
! sysctl -w net.ipv4.tcp_syncookies=0 #(turn off SYN cookie)

In [ ]:
# the following command shows all the halfopen connections
! netstat -na | grep SYN-RECV

In [ ]:
# send SYN PACKET To the server and check for SYN-RECV status
#while True:
pkt = IP(dst=f"{host}")/TCP(dport=80)
reply = sr1(pkt)

In [ ]:
reply.show()

## Create ping request
- IP packet with ICMP echo request and some payload
- send the packet and capture it with wireshark

In [76]:
host = 'google.com'

In [77]:
# using spoofed IP
payload = "tell me if you get this?"
ping = IP(dst=f"{host}") / ICMP() / payload 

In [78]:
ping.show()

###[ IP ]###
  version   = 4
  ihl       = None
  tos       = 0x0
  len       = None
  id        = 1
  flags     = 
  frag      = 0
  ttl       = 64
  proto     = icmp
  chksum    = None
  src       = 192.168.35.128
  dst       = Net("google.com/32")
  \options   \
###[ ICMP ]###
     type      = echo-request
     code      = 0
     chksum    = None
     id        = 0x0
     seq       = 0x0
     unused    = b''
###[ Raw ]###
        load      = b'tell me if you get this?'



In [79]:
send(ping)

.
Sent 1 packets.


In [80]:
# you'll never receive the ping response, if your spoof the source destination
reply = sr1(ping)
reply.show()

Begin emission

Finished sending 1 packets

Received 1 packets, got 1 answers, remaining 0 packets
###[ IP ]###
  version   = 4
  ihl       = 5
  tos       = 0x0
  len       = 52
  id        = 62513
  flags     = 
  frag      = 0
  ttl       = 128
  proto     = icmp
  chksum    = 0x89e6
  src       = 142.250.73.142
  dst       = 192.168.35.128
  \options   \
###[ ICMP ]###
     type      = echo-reply
     code      = 0
     chksum    = 0x1e53
     id        = 0x0
     seq       = 0x0
     unused    = b''
###[ Raw ]###
        load      = b'tell me if you get this?'



In [ ]:
'tell me' in reply['Raw'].load.decode('utf-8')

## Exercise

- Spoof src IP and ping some IP in the network you own!
- 